# Energy minimization example

This example uses the Inexact Newton CG algorithm to minimize a non-quadratic energy functional

$$ E(u) = \frac{1}{2} \int_\Omega k(u) \nabla u \cdot \nabla u d\boldsymbol{x} + \frac{1}{2} \int_{\partial\Omega} \beta u^2 d\boldsymbol{s} - \int_\Omega f u d\boldsymbol{x}, $$

where $\Omega$ is the unit circle.

In [ ]:
# Initialization
import matplotlib.pyplot as plt
%matplotlib inline

import dolfin as dl
import ufl
import math
import numpy as np
import logging
from unconstrainedMinimization import InexactNewtonCG

logging.getLogger('FFC').setLevel(logging.WARNING)
logging.getLogger('UFL').setLevel(logging.WARNING)
dl.set_log_active(False)

mesh = dl.Mesh("../Problem3/circle.xml")

# Define the finite element space
Vh = dl.FunctionSpace(mesh, "Lagrange", 2)

u = dl.Function(Vh)
u_hat = dl.TestFunction(Vh)
u_tilde = dl.TrialFunction(Vh)

# Define the energy functional
f = dl.Expression("100*exp(-100*(x[0]*x[0] + x[1]*x[1]))",degree=5)
beta = dl.Constant(100)
k1 = dl.Constant(.1)
k2 = dl.Constant(1.)

E = dl.Constant(.5)*beta*u*u*ufl.ds + \
    dl.Constant(.5)*(k1 + k2*u*u)*ufl.inner(ufl.grad(u), ufl.grad(u))*ufl.dx - \
    f*u*ufl.dx

grad = dl.derivative(E, u, u_hat)

H = dl.derivative(grad, u, u_tilde)
    
    
solver = InexactNewtonCG()
solver.parameters["rel_tolerance"] = 1e-6
solver.parameters["abs_tolerance"] = 1e-9
solver.parameters["gdu_tolerance"] = 1e-18
solver.parameters["max_iter"] = 1000
solver.parameters["c_armijo"] = 1e-5
solver.parameters["print_level"] = 1
solver.parameters["max_backtracking_iter"] = 10

solver.solve(E, u, grad, H)

dl.plot(u)
plt.show()

 It          Energy           ||g||          (g,du)           alpha          tol_cg   cg_it
  0   -2.189399e+00    8.786318e-01   -2.970296e+01    1.250000e-01    5.000000e-01       1


Copyright (c) 2016, The University of Texas at Austin & University of California, Merced.

All Rights reserved.
See file COPYRIGHT for details.

This file is part of the hIPPYlib library. For more information and source code availability see https://hippylib.github.io.

hIPPYlib is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License (as published by the Free Software Foundation) version 2.0 dated June 1991.